In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
news_df = pd.read_excel('./DH(뉴스+타겟)222.xlsx', index_col=0)
news_df.head()

,predict_date,oil_news,article_text,대한종가,대한타겟
0,2022-01-02,코로나19가 2년가까이 지속되면서 대면서비스업을 중심으로 한 내수경기의 부진은 더욱...,2022년에도 국제선 여객 정상화는 어렵다는 판단이며 이에 따라 항공화물 시황은 호...,29500,0
1,2022-01-03,2021년 12월 충청지역 소비자물가동향 충북과 충남의 물가 상승률이 4%대를 기록...,26일 오후 경기도 의정부체육관에서 열린 2021 의정부 도드람컵 프로배구대회 순위...,29500,0
2,2022-01-04,서울 시내 주유소 모습. 주요 에너지기관들은 2022년에도 국제 유가가 상승세를 이...,사천공항 경남도는 진에어가 오는 28일부터 김포-사천 노선에 신규 취항한다고 4일 ...,29100,0
3,2022-01-05,"저작권자 서울경제, 무단 전재 및 재배포 금지 미국 뉴욕증시의 주요 지수가 국채금리...",인천 중구 영종국제도시 대한항공 항공정비단지 부지. 인천시 제공 대한항공이 인천 영...,28900,0
4,2022-01-06,美 남미 등 수입처 다변화 안정적 물량 확보는 어려워 코로나19 여파로 원유 가격이...,"인천시-대한항공, 항공정비산업 발전 업무협약 체결 대한항공, 3346억 투자 202...",28450,0


In [ ]:
model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

texts = news_df['article_text'].tolist()

In [ ]:

embedded_news = []
for news in news_df['article_text']:
    # 문장 토큰화 및 패딩
    inputs = tokenizer.encode_plus(
        news,
        add_special_tokens=True,
        padding="max_length",
        max_length=300,
        truncation=True,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.squeeze(0)
        embedded_news.append(embeddings.tolist())

In [ ]:
news_df['kcbert_embedding'] = embedded_news

In [ ]:
news_df['kcbert_embedding']

0      [[-0.14264455437660217, 0.05831192433834076, 0...
1      [[-0.15551824867725372, -0.03494337573647499, ...
2      [[-0.6354258060455322, -0.365503191947937, 0.9...
3      [[1.2611210346221924, -1.2480064630508423, 0.5...
4      [[-1.170987844467163, -0.04770953953266144, 0....
                             ...                        
479    [[-0.8856520056724548, 0.06582590192556381, 0....
480    [[-0.7657447457313538, 1.1281311511993408, -2....
481    [[-0.12134005129337311, -0.26261863112449646, ...
482    [[0.152285635471344, -0.14103375375270844, 0.3...
483    [[-0.13556396961212158, 0.7741327285766602, 0....
Name: kcbert_embedding, Length: 484, dtype: object